In [1]:
import pandas as pd
import json
import numpy as np
from tqdm import tqdm

import data_process as d

from DataProcessing import RecsData as recs

# Traitements des données
Les données sont des fichiers textes sous la forme **JSON**. Il existe deux fichiers distinct : **offering** et **reviews**.

## Présentation de Offering
Offering contient tous les hôtels valables à notre disposition.

In [5]:
df_offering = pd.read_csv('../data/optimiser/offering.csv')
df_offering.drop('Unnamed: 0', axis=1, inplace=True)
df_offering.head()

,hotel_class,region_id,url,phone,details,type,id,name,address.region,address.street-address,address.postal-code,address.locality
0,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,113317,Casablanca Hotel Times Square,NY,147 West 43rd Street,10036,New York City
1,5.0,32655,http://www.tripadvisor.com/Hotel_Review-g32655...,NaN,NaN,hotel,76049,Four Seasons Hotel Los Angeles at Beverly Hills,CA,300 S Doheny Dr,90048,Los Angeles
2,3.5,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,99352,Hilton Garden Inn Times Square,NY,790 Eighth Avenue,10019,New York City
3,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,93589,The Michelangelo Hotel,NY,152 West 51st Street,10019,New York City
4,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,217616,The Muse Hotel New York,NY,130 West 46th Street,10036,New York City


## Présentation de reviews
Reviews contient les appréciations des clients dans différents hôtels.

In [6]:
reviews = recs("../data/review.txt")
# prc ou pourcentage limite le nombre de donnée en lecture
reviews_json = reviews.to_json(prc=0.1)
new_reviews = pd.json_normalize(reviews_json, meta=['ratings', 'author'])
new_reviews.head()

Initialize the dataset


878561it [00:32, 26927.39it/s]
100%|█████████████████████████████████████████████████████████████████████████| 87856/87856 [00:04<00:00, 19622.04it/s]


,title,text,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile,ratings.service,ratings.cleanliness,...,ratings.business_service_(e_g_internet_access),ratings.location,author.username,author.num_cities,author.num_helpful_votes,author.num_reviews,author.location,author.id,ratings.sleep_quality,author.num_type_reviews
0,“Great place to stay”,I would recommend the Castle Inn to anyone loo...,January 2009,119658,0,"January 22, 2009",23804385,False,4.0,5.0,...,4.0,4.0,akhgyeiks,2.0,1.0,2.0,"Rochester, NY",651F2DB2FB0150F8C7EDA0BB853950D1,NaN,NaN
1,“Better than expected...”,My husband booked a night at Fort Rapids witho...,March 2012,101653,1,"April 16, 2012",127984679,False,3.0,4.0,...,NaN,4.0,andvium,14.0,18.0,26.0,,E4C989EB79982B555342EE3359D7AA92,4.0,8.0
2,“Ótima Localização!”,"Hotel super bem localizado, podendo fazer tudo...",October 2011,208454,0,"October 31, 2011",119969149,True,4.0,4.0,...,NaN,5.0,VanGuti,6.0,4.0,6.0,,E53B60F7579FE74007AD9523F9E21B17,NaN,6.0
3,“Excellent place to stay. Close to the airport...,This is an excellent place to stay when on a b...,June 2011,217635,0,"August 1, 2011",116099442,False,5.0,5.0,...,NaN,NaN,ParagSohoni,6.0,4.0,8.0,"Pune, India",F90DA8A551AAC8424A52C366C3120CC6,5.0,8.0
4,“Average Embassy Suites”,I have to say that this is a an average to bel...,May 2012,94122,0,"May 31, 2012",131013296,False,2.0,3.0,...,NaN,3.0,fletch608,22.0,21.0,36.0,Pittsburgh,8282D1AC1D24DFEC7B0102153907DF00,3.0,19.0


Voici un extrait de valeur de la clé **author** par exemple.

In [90]:
new_reviews.filter(regex='author')[new_reviews['author.num_reviews'].isnull()]

,author.username,author.num_cities,author.num_helpful_votes,author.num_reviews,author.num_type_reviews,author.id,author.location
28,,NaN,NaN,NaN,NaN,,
38,,NaN,NaN,NaN,NaN,,
40,,NaN,NaN,NaN,NaN,,
44,,NaN,NaN,NaN,NaN,,
47,,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...
87776,,NaN,NaN,NaN,NaN,,
87806,,NaN,NaN,NaN,NaN,,
87818,,NaN,NaN,NaN,NaN,,
87823,,NaN,NaN,NaN,NaN,,


## Optimisation de Reviews

In [8]:
df_reviews = pd.read_csv('../data/optimiser/complete_reviews.csv')
df_reviews.head()

,Unnamed: 0,title,text,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile,ratings.service,...,ratings.check_in_front_desk,ratings.business_service_(e_g_internet_access),ratings.location,author.username,author.num_cities,author.num_helpful_votes,author.num_reviews,author.num_type_reviews,author.id,author.location
0,0,0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,December 2012,93338,0,"December 17, 2012",147643103,False,...,5.0,5.0,Papa_Panda,22,12,29,24,8C0B42FF3C0FA366A21CFD785302A032,Gold Coast,NaN
1,1,0,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...",December 2012,93338,0,"December 17, 2012",147639004,False,...,5.0,5.0,Maureen V,2,2,E3C85CA9DBBBC77E0DB534ABE93E4713,"Sydney, New South Wales, Australia",NaN,NaN,NaN
2,2,0,“Great Stay”,This is a great property in Midtown. We two di...,December 2012,1762573,0,"December 18, 2012",147697954,False,...,4.0,4.0,vuguru,12,17,14,14,FB1032DECE1162CB3556D05F278AAFFD,Houston,NaN
3,3,0,“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,August 2012,1762573,0,"December 17, 2012",147625723,False,...,5.0,5.0,Hotel-Designer,5,26,5,5,EC3E275EE7590694889C8C7EE0D13961,"Laguna Beach, CA",NaN
4,4,0,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,December 2012,1762573,0,"December 17, 2012",147612823,False,...,5.0,5.0,JamesE339,34,65,104,49,BA524A238B1171206691A6CC3F28F266,"Saint Louis, Missouri",NaN
